### Creation of the environment

In [ ]:
%tensorflow_version 2.x
!pip3 install --upgrade pip
#!pip install -qU t5
!pip3 install git+https://github.com/google-research/text-to-text-transfer-transformer.git #extra_id_x support

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

#Set the base dir(Google cloud bucket)
BASE_DIR = "gs://bucket_code_completion" 

if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
ON_CLOUD = True


if ON_CLOUD:
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

     |████████████████████████████████| 1.5MB 6.9MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Cloning https://github.com/google-research/text-to-text-transfer-transformer.git to /tmp/pip-req-build-m7v6dzvw
  Running command git clone -q https://github.com/google-research/text-to-text-transfer-transformer.git /tmp/pip-req-build-m7v6dzvw
     |████████████████████████████████| 361 kB 6.4 MB/s 
     |████████████████████████████████| 65 kB 3.4 MB/s 
     |████████████████████████████████| 1.2 MB 8.7 MB/s 
     |████████████████████████████████| 3.4 MB 13.0 MB/s 
     |████████████████████████████████| 3.8 MB 36.4 MB/s 
     |████████████████████████████████| 1.8 MB 73.9 MB/s 
     |████████████████████████████████| 883 kB 78.1 MB/s 
     |████████████████████████████████| 2.9 MB 85.9 MB/s 
  Created wheel for t5: filename=t5-0.8.1-py3-none-any.whl size=221272 sha256=3b06ab397807c6d3cc1b50de4cb9560123ed2a0c9a2326c7

Instructions for updating:
non-resource variables are not supported in the long term


### Loading of tsv files
With this script you can load each tsv file for finetuning.
Please be sure that the path to all tsv files are correct

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_java_construct = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_java_construct.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/test_java_construct.tsv',
}

num_nq_examples_java_construct = dict(train=750000, validation=106237)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_android_construct = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_android_construct.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/test_android_construct.tsv',
}

num_nq_examples_android_construct = dict(train=750000, validation=100536)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_java_block = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_java_block.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/test_java_block.tsv',
}

num_nq_examples_java_block = dict(train=298470, validation=40008)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_android_block = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_android_block.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/test_android_block.tsv',
}

num_nq_examples_android_block = dict(train=204580, validation=26978)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_java_token = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_java_token.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/test_java_token.tsv',
}

num_nq_examples_java_token = dict(train=750000, validation=219486)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_android_token = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_android_token.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/test_android_token.tsv',
}

num_nq_examples_android_token = dict(train=750000, validation=200504)

### Preprocess of the dataset
In this step we preprocess the dataset.  
You have to change the path to vocab files (*vocab_model_path* and *vocab_path*)
We're going to preprocess all the tsv file so that T5 can use them for evaluation.

In [ ]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary


# # Set the path of sentencepiece model and vocab files
# # Must be the same used for the pre-trained phase
vocab_model_path = 'gs://bucket_code_completion/T5_extension/code.model'
vocab_path = 'gs://bucket_code_completion/T5_extension/code.vocab'


TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask


def get_default_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, 100)

DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True, required=False),

    "targets": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True)
}

JAVA CONSTRUCT

In [ ]:
def nq_java_construct(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

   # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path_java_construct[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(nq_java_construct("train").take(5)):
  print(ex)

A few raw train examples...
{'input': b'public void updateLockdownExceptions(ManagedObjectReference _this, String[] users) throws AuthMinimumAdminPermission, RemoteException, RuntimeFault, UserNotFound { Argument[] params = new Argument[2]; params[0] = new Argument("_this", "ManagedObjectReference", _this); params[1] = new Argument("users", "String[]", users); getWsc().invoke( <extra_id_0>); }', 'output': b'"UpdateLockdownExceptions", params, null'}
{'input': b'@Override public Collection<AgentProjectInfo> collectDependencies(String folder) { if ( <extra_id_0>){ devDependencies = findDevDependencies(folder); } File yarnLock = new File(folder + fileSeparator + YARN_LOCK); boolean yarnLockFound = yarnLock.isFile(); Collection<DependencyInfo> dependencies = new ArrayList<>(); if (yarnLockFound){ dependencies = parseYarnLock(yarnLock); } else { npmLsFailureStatus = true; } return getSingleProjectList(dependencies); }', 'output': b'!includeDevDependencies'}
{'input': b'@Override public Coll

In [ ]:
def java_construct_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['JAVA_CONSTRUCT:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('java_construct')
t5.data.TaskRegistry.add(
    "java_construct",
    dataset_fn=nq_java_construct,
    splits=["train", "validation"],
    text_preprocessor=[java_construct_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_java_construct
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("java_construct")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'JAVA_CONSTRUCT:public static File file(File directory, String... names) { Assert.notNull(directory, "directorydirectory must not be null"); if (ArrayUtil.isEmpty(names)) { return directory; } File file = directory; for (String name : names) { if ( <extra_id_0>) { file = file(file, name); } } return file; }', 'inputs': array([    3,  7641,    15,  5071, 17558,    56,  4569,    48,   181,
         191,     5,   104,  1364,     9,    26,     4,     4,     4,
        1886,     8,     7,   865,     4,  2623,     5,  2914,     9,
          32,  2914,  2914,   848,   153,   198,    30,    46,    21,
          17,   368,   306,     4,   280,     5,  3381,     8,     8,
           7,    14,  1364,    13,     6,   181,   191,    11,  1364,
          13,    50,    17,    31,   103,    58,  1886,     8,     7,
          21,    17, 32099,     8,     7,   191,    11,   191,     5,
         296,     9,   103,    10,     6,     6,    14

JAVA TOKEN

In [ ]:
def nq_java_token(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_java_token[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_java_token("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'private void emitSelectById(JavaWriter <extra_id_0> logger.d("emitSelectById"); javaWriter.beginMethod(getTargetClass(), $$GET_OBJECT_BY_ID, EnumSet.of(PUBLIC, STATIC), "long", "id", "SQLiteDatabase", "db") .emitStatement("Cursor cursor = db.rawQuery(\\"SELECT * FROM %s WHERE %s = id\\", null)", getTableName(), idColumn.getColumnName()) .emitStatement("%s value = %s(cursor, db).get(0)", getTargetClass(), $$MAP_OBJECT_FUNCTION) .emitStatement("cursor.close()") .emitStatement("return value") .endMethod(); }', 'output': b'javaWriter) throws IOException {'}
{'input': b'private void emitSelectById(JavaWriter javaWriter) throws IOException { logger.d("emitSelectById" <extra_id_0> javaWriter.beginMethod(getTargetClass(), $$GET_OBJECT_BY_ID, EnumSet.of(PUBLIC, STATIC), "long", "id", "SQLiteDatabase", "db") .emitStatement("Cursor cursor = db.rawQuery(\\"SELECT * FROM %s WHERE %s = id\\", null)", getTableName(), idColumn.getColumnName()) .emitStatement("%s

In [ ]:
def java_token_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['JAVA_TOKEN:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('java_token')
t5.data.TaskRegistry.add(
    "java_token",
    dataset_fn=nq_java_token,
    splits=["train", "validation"],
    text_preprocessor=[java_token_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_java_token
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("java_token")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


A few preprocessed training examples...


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


{'inputs_pretokenized': b'JAVA_TOKEN:public boolean isRenderable() { if (isRenderable == null) { String extension = getNameExtension(); isRenderable = isTyped() || (isText() && (HTML.equals(extension) )); } return isRenderable <extra_id_0> }', 'inputs': array([    3,  7641,    15,  2591,    56,  4569,    45,    69, 22326,
          16,     7,    21,    17,   112, 22326,    40,    30,     8,
           7,    26,  1884,    11,  1975,   867,    18,    69, 22326,
          11,    69,  4947,    16,     3,     2,    17,   112,   204,
          16,    91,    17,  3536,     4,   117,     5,  3645,     8,
           3,    79,     6,    14,    69, 22326, 32099,     6,     1],
      dtype=int32), 'targets_pretokenized': b';', 'targets': array([ 3, 13,  1], dtype=int32)}
{'inputs_pretokenized': b'JAVA_TOKEN:public String getRequestLogMessage(Throwable e, HttpServletRequest <extra_id_0> Throwable cause = getRootCause(e); String exceptionName = cause.getClass().getSimpleName(); return getRequestLogM

JAVA BLOCK

In [ ]:
def nq_java_block(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_java_block[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_java_block("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public String capitalizeWords(final Object target) { if (target == null) <extra_id_0> return StringUtils.capitalizeWords(target); }', 'output': b'{ return null; }'}
{'input': b'@SuppressWarnings("unchecked") public <T extends CMAResource> T setEnvironmentId(String environmentId) { if (getSystem().getEnvironment() == null) <extra_id_0> getSystem().environment.setId(environmentId); return (T) this; }', 'output': b'{ getSystem().environment = new CMALink(CMAType.Environment); }'}
{'input': b'@Deprecated public static byte[] convertLongToVarInt(long value) { ByteBuffer longBB = ByteBuffer.allocate(EthereumUtil.LONG_SIZE); longBB.putLong(value); byte[] result = longBB.array(); int leadingZeros=0; for (int i=0;i<result.length;i++) { if (result[i]==0) <extra_id_0> else { break; } } return Arrays.copyOfRange(result, leadingZeros, result.length); }', 'output': b'{ leadingZeros++; }'}
{'input': b'@Deprecated public static byte[] convertLongToVarInt(long va

In [ ]:
def java_block_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['JAVA_BLOCK:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('java_block')
t5.data.TaskRegistry.add(
    "java_block",
    dataset_fn=nq_java_block,
    splits=["train", "validation"],
    text_preprocessor=[java_block_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_java_block
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("java_block")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


A few preprocessed training examples...


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


{'inputs_pretokenized': b'JAVA_BLOCK:@Override public V g(K key) throws PageException { V v = map.get(key); if (v == NULL) return null; if (v == null) <extra_id_0> return v; }', 'inputs': array([    3,  7641,    15,  3517,    56,  2098,    27,    12,   862,
         841,     5,   390,   145,     8,    42,  3031,    38,     7,
         862,   284,    11,   342,     4,    33,     5,   158,    10,
          21,    17,   291,    40,  5614,     8,    14,    30,    13,
          21,    17,   291,    40,    30,     8, 32099,    14,   284,
          13,     6,     1], dtype=int32), 'targets_pretokenized': b'throw invalidKey(this, key, false);', 'targets': array([  78, 2723,  133,    5,   75,    9,  145,    9,   76,   10,    1],
      dtype=int32)}
{'inputs_pretokenized': b'JAVA_BLOCK:public void set(String paramter, Object value) { ChangeLogParameter param = findParameter(paramter, null); if (param == null) <extra_id_0> }', 'inputs': array([    3,  7641,    15,  3517,    56,  4569,    20,    5

ANDROID CONSTRUCT

In [ ]:
def nq_android_construct(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

   # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path_android_construct[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(nq_android_construct("train").take(5)):
  print(ex)

A few raw train examples...
{'input': b'private void writeToFile(final String content) { if ( <extra_id_0>) { return; } try { System.err.println(content); File f = new File(OUTPUT_FILE); f.delete(); FileWriter fstream = new FileWriter(OUTPUT_FILE); BufferedWriter out = new BufferedWriter(fstream); out.write(content); out.close(); } catch (Exception e) { e.printStackTrace(); } }', 'output': b'!WRITE_TO_FILE'}
{'input': b'private void writeToFile(final String content) { if (!WRITE_TO_FILE) { return; } try { System.err.println(content); File f = new File(OUTPUT_FILE); f.delete(); FileWriter fstream = new FileWriter(OUTPUT_FILE); BufferedWriter out = new BufferedWriter(fstream); out.write(content); out.close(); } catch ( <extra_id_0>) { e.printStackTrace(); } }', 'output': b'Exception e'}
{'input': b'private void writeToFile(final String content) { if (!WRITE_TO_FILE) { return; } try { System.err.println(content); File f = new File( <extra_id_0>); f.delete(); FileWriter fstream = new FileW

In [ ]:
def android_construct_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['ANDROID_CONSTRUCT:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('android_construct')
t5.data.TaskRegistry.add(
    "android_construct",
    dataset_fn=nq_android_construct,
    splits=["train", "validation"],
    text_preprocessor=[android_construct_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_android_construct
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("android_construct")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


A few preprocessed training examples...


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


{'inputs_pretokenized': b'ANDROID_CONSTRUCT:private void buildCursorMapForExistingCollaborators(Collaborator collaborator) { if (collaborator == null) { return; } IndexReference ref = (IndexReference) mCursors.get(collaborator.getSessionId()); if (ref == null) { return; } Set<Collaborator> row = new HashSet<>(); mRowToCollaborators.put( <extra_id_0>); row.add(collaborator); }', 'inputs': array([    3, 16446,    15,  5071, 17558,    56,  8797,    20,   501,
        1123,   100,   203,  4001, 18991,    22,     5, 18991,     3,
       19359,     8,     7,    21,    17, 19359,    40,    30,     8,
           7,    14,    13,     6,     3,   163,   478,  1359,    11,
          17,   163,   478,     8,  8808,    22,     4,    33,     5,
       19359,     4, 13816,    39,    21,    17,  1391,    40,    30,
           8,     7,    14,    13,     6,   300,    25, 18991,    29,
         722,    11,    24,   978,   447,    54,   634,   129, 18991,
          22,     4,   120,     5, 32099,    10, 

ANDROID TOKEN

In [ ]:
def nq_android_token(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_android_token[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_android_token("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public boolean initialize <extra_id_0> if (bluetoothManager == null) { bluetoothManager = (BluetoothManager) getSystemService(Context.BLUETOOTH_SERVICE); if (bluetoothManager == null) { Log.e(TAG, "Unable to initialize BluetoothManager."); return false; } } adapter = bluetoothManager.getAdapter(); if (adapter == null) { Log.e(TAG, "Unable to obtain a BluetoothAdapter."); return false; } return true; }', 'output': b'() {'}
{'input': b'public boolean initialize() { if (bluetoothManager == <extra_id_0> bluetoothManager = (BluetoothManager) getSystemService(Context.BLUETOOTH_SERVICE); if (bluetoothManager == null) { Log.e(TAG, "Unable to initialize BluetoothManager."); return false; } } adapter = bluetoothManager.getAdapter(); if (adapter == null) { Log.e(TAG, "Unable to obtain a BluetoothAdapter."); return false; } return true; }', 'output': b'null) {'}
{'input': b'public boolean initialize() { if (bluetoothManager == null) { bluetoothManager = (Blu

In [ ]:
def android_token_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['ANDROID_TOKEN:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('android_token')
t5.data.TaskRegistry.add(
    "android_token",
    dataset_fn=nq_android_token,
    splits=["train", "validation"],
    text_preprocessor=[android_token_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_android_token
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("android_token")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'ANDROID_TOKEN:public List<GalleryObject> loadStoredGalleryObjects(GalleryObject parent) { if (parent == null) { return null; } if (!parent.equals(mStoredGalleryObject)) { return <extra_id_0> } return mGalleryChildObjects; }', 'inputs': array([    3, 16446,    15,  2591,    56,  4569,    85,    25, 11060,
          96,    29,   551,  4438, 11060,  1620,     5, 11060,    96,
         340,     8,     7,    21,    17,   614,    40,    30,     8,
           7,    14,    30,    13,     6,    21,   124,   614,     4,
         117,     5,    87,  4438, 11060,    96,     8,     8,     7,
          14, 32099,     6,    14,    54, 11060,   995,  1620,    13,
           6,     1], dtype=int32), 'targets_pretokenized': b'null;', 'targets': array([30, 13,  1], dtype=int32)}
{'inputs_pretokenized': b'ANDROID_TOKEN:protected void onPostCreate(Bundle savedInstanceState) { super.onPostCreate(savedInstanceState <extra_id_0> setContentView(

ANDROID BLOCK

In [ ]:
def nq_android_block(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_android_block[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_android_block("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public void onStackEmpty() { cancelAutoLove(); if (progressDialog == null || !progressDialog.isShowing()) <extra_id_0> }', 'output': b'{ progressDialog = ProgressDialog.show(this, "Out of Cats", "Searching for more, Please wait..."); startLoading(); }'}
{'input': b'private static List<HeroAndAdvantages> loadHeroes(SQLiteDatabase db) { List<HeroAndAdvantages> heroes = new ArrayList<>(); Cursor c = db.rawQuery("SELECT * FROM Heroes", null); c.moveToFirst(); while (!c.isAfterLast()) <extra_id_0> c.close(); return heroes; }', 'output': b'{ heroes.add(new HeroAndAdvantages(c)); c.moveToNext(); }'}
{'input': b'private String getLinkByRelation(String relation) { for (Link l : link) { if (l.getRel().equals(relation)) <extra_id_0> } return null; }', 'output': b'{ return l.getUri(); }'}
{'input': b'public static boolean isBundleValid(final Bundle bundle) { if (null == bundle) <extra_id_0> if (bundle.getInt(BUNDLE_EXTRA_INT_VERSION_CODE, -1) == -1) { return

In [ ]:
def android_block_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['ANDROID_BLOCK:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('android_block')
t5.data.TaskRegistry.add(
    "android_block",
    dataset_fn=nq_android_block,
    splits=["train", "validation"],
    text_preprocessor=[android_block_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_android_block
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("android_block")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


A few preprocessed training examples...


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


{'inputs_pretokenized': b'ANDROID_BLOCK:public NotificationItemAdapter(Context context, List<NotificationItem> objects) { super(context, 0, objects); this.sort(new Comparator<NotificationItem>() { @Override public int compare(NotificationItem lhs, NotificationItem rhs) <extra_id_0> }); }', 'inputs': array([    3, 16446,    15,  3517,    56,  4569,  1895,   169,   442,
           5,    92,   130,     9,    85,    25,   836,   169,    29,
        2808,     8,     7,    52,     5,   201,     9,   312,  2808,
          10,    23,     4,  1703,     5,    74,  9719,    25,   836,
         169,    29,    16,     7,    19,    27,    12,    35,  1402,
           5,   836,   169,  5633,     9,  1895,   169,  3613,     8,
       32099,     6,    10,     6,     1], dtype=int32), 'targets_pretokenized': b'{ return rhs.getDate().compareTo(lhs.getDate()); }', 'targets': array([   7,   14, 3613,    4, 5765,   37, 1365,    5, 8804,    4, 5765,
         39,    6,    1], dtype=int32)}
{'inputs_pretokeniz

### Evaluation
You can run the evaluation using the following cells.  
Please set the correct path of the variable *MODEL_DIR* (the path to save the pretrained model in)


In [ ]:
def _rate_num_input_examples(task):
  if "train" in task.splits:
    return float(task.num_input_examples("train"))
  elif "validation" in task.splits:
    return float(task.num_input_examples("validation"))
  else:
    raise ValueError("Task %s does not have a train or validation split." % (task.name))


t5.data.MixtureRegistry.remove("all_tasks")
t5.data.MixtureRegistry.add(
    "all_tasks",
    ["java_construct", "java_token", "java_block", "android_construct", "android_token", "android_block"],
    default_rate=_rate_num_input_examples
     #default_rate=1.0
)

In [ ]:
from mesh_tensorflow.transformer.learning_rate_schedules import slanted_triangular

MODEL_SIZE = "small" 

# Set the folder where the checkpoints and all the others information will be writed
MODEL_DIR = 'gs://bucket_code_completion/T5_extension/finetuning'

model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)

model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    learning_rate_schedule = slanted_triangular,
    sequence_length={"inputs": 256, "targets": 256},
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [ ]:
# we used model.predict function (setting beam_size)

vocabulary_predict=get_default_vocabulary()

model.predict(input_file='gs://bucket_code_completion/T5_extension/finetuning/predict/inputs.txt', output_file='gs://bucket_code_completion/T5_extension/finetuning/predict/predictions.txt',
              checkpoint_steps=-1, beam_size=1, temperature=0.0, keep_top_k=-1, vocabulary=vocabulary_predict)

INFO:root:system_path_file_exists:gs://bucket_comment_completion/Matteo/finetuning/operative_config.gin
ERROR:root:Path not found: gs://bucket_comment_completion/Matteo/finetuning/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://bucket_comment_completion/Matteo/finetuning', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.112.178.66:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.112.178.66:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.112.178.66:8470', '_evaluation_master': 'grpc://10.112.178